In [1]:
import torch
from torch import nn

torch.__version__

'2.2.1+cu118'

In [2]:
# Setup device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"

device

'cuda'

# Preparing Data

### Getting Data

In [4]:
import pandas as pd
pd.set_option('display.max_columns',200)
pd.set_option('display.max_rows',200)

In [5]:
from sklearn.datasets import load_files       
from tensorflow.keras.utils import to_categorical
import numpy as np
from glob import glob

# define function to load train, test, and validation datasets
def load_dataset(path):
    data = load_files(path)
    dog_files = np.array(data['filenames'])
    dog_targets = to_categorical(np.array(data['target']), 133)  # Updated to use to_categorical
    return dog_files, dog_targets

# load train, test, and validation datasets
train_files, train_targets = load_dataset(r'C:\Users\A\Downloads\dogImages\train')
valid_files, valid_targets = load_dataset(r'C:\Users\A\Downloads\dogImages\valid')
test_files, test_targets = load_dataset(r'C:\Users\A\Downloads\dogImages\test')

# load list of dog names
dog_names = [item[38:-1] for item in sorted(glob(r'C:\Users\A\Downloads\dogImages\train\*/'))]

# print statistics about the dataset
print('There are %d total dog categories.' % len(dog_names))
print('There are %s total dog images.\n' % len(np.hstack([train_files, valid_files, test_files])))
print('There are %d training dog images.' % len(train_files))
print('There are %d validation dog images.' % len(valid_files))
print('There are %d test dog images.' % len(test_files))


There are 133 total dog categories.
There are 8351 total dog images.

There are 6680 training dog images.
There are 835 validation dog images.
There are 836 test dog images.


In [7]:
data = load_files(r'C:\Users\A\Downloads\dogImages\test')

In [8]:
data['target']

array([ 56,  58, 114,  18,  67,  20,  42,  51,  96,  50,   1,  18,  74,
       132,  70,  13, 122,  31,  90,  41,  97,  68,  55,  94,  35, 124,
        18,  18,  91, 112,   4, 108,  28,  22, 105,  79,  34,  38,   3,
       117,  80, 129,  41,  26,  60,   1, 100,  89,  87,   7,  32,  66,
        32,  13,  34, 118, 101, 103,  29,  96,  10, 102,  33,  95,  99,
        27, 132, 121,  42,  37,  87,  57, 100,  58,  70,  60,  29, 123,
        37,  67, 127, 102,  39,  81,  92,  86,  64,  45,   5,  36,  93,
        14, 130,  69,  21,  19, 128,  69,  34, 108,   7,  10,  58,  23,
        71, 114,  51,  56,  83,  86,  99,  60,  43,  11,  70,  10, 118,
        82,  27,  43,  75,   0, 131,   0,  36, 109,  80,  94,  43,  20,
        47, 117,  26, 117,  69,  53,  19,  55,  53,  49,  64,  74,  47,
        55,   4,   2,  66,  43,  87,   6, 124, 100,  15,  82,  91,   1,
        67,   9,  85, 112,  97,  28, 122,  87,  34,   0, 130,  97, 115,
        90,  71, 120, 117,  59,  35,  72, 119,  35, 111,  62,  4

In [9]:
data['filenames']

array(['C:\\Users\\A\\Downloads\\dogImages\\test\\057.Dalmatian\\Dalmatian_04056.jpg',
       'C:\\Users\\A\\Downloads\\dogImages\\test\\059.Doberman_pinscher\\Doberman_pinscher_04156.jpg',
       'C:\\Users\\A\\Downloads\\dogImages\\test\\115.Papillon\\Papillon_07480.jpg',
       'C:\\Users\\A\\Downloads\\dogImages\\test\\019.Bedlington_terrier\\Bedlington_terrier_01363.jpg',
       'C:\\Users\\A\\Downloads\\dogImages\\test\\068.Flat-coated_retriever\\Flat-coated_retriever_04703.jpg',
       'C:\\Users\\A\\Downloads\\dogImages\\test\\021.Belgian_sheepdog\\Belgian_sheepdog_01548.jpg',
       'C:\\Users\\A\\Downloads\\dogImages\\test\\043.Canaan_dog\\Canaan_dog_03072.jpg',
       'C:\\Users\\A\\Downloads\\dogImages\\test\\052.Clumber_spaniel\\Clumber_spaniel_03715.jpg',
       'C:\\Users\\A\\Downloads\\dogImages\\test\\097.Lakeland_terrier\\Lakeland_terrier_06518.jpg',
       'C:\\Users\\A\\Downloads\\dogImages\\test\\051.Chow_chow\\Chow_chow_03664.jpg',
       'C:\\Users\\A\\Downloads\

In [10]:
dog_names

['01.Affenpinscher',
 '02.Afghan_hound',
 '03.Airedale_terrier',
 '04.Akita',
 '05.Alaskan_malamute',
 '06.American_eskimo_dog',
 '07.American_foxhound',
 '08.American_staffordshire_terrier',
 '09.American_water_spaniel',
 '10.Anatolian_shepherd_dog',
 '11.Australian_cattle_dog',
 '12.Australian_shepherd',
 '13.Australian_terrier',
 '14.Basenji',
 '15.Basset_hound',
 '16.Beagle',
 '17.Bearded_collie',
 '18.Beauceron',
 '19.Bedlington_terrier',
 '20.Belgian_malinois',
 '21.Belgian_sheepdog',
 '22.Belgian_tervuren',
 '23.Bernese_mountain_dog',
 '24.Bichon_frise',
 '25.Black_and_tan_coonhound',
 '26.Black_russian_terrier',
 '27.Bloodhound',
 '28.Bluetick_coonhound',
 '29.Border_collie',
 '30.Border_terrier',
 '31.Borzoi',
 '32.Boston_terrier',
 '33.Bouvier_des_flandres',
 '34.Boxer',
 '35.Boykin_spaniel',
 '36.Briard',
 '37.Brittany',
 '38.Brussels_griffon',
 '39.Bull_terrier',
 '40.Bulldog',
 '41.Bullmastiff',
 '42.Cairn_terrier',
 '43.Canaan_dog',
 '44.Cane_corso',
 '45.Cardigan_welsh_c

In [11]:
import torchvision.models as models

#ResNet50_model
ResNet50_model = models.resnet50(pretrained = True)

# Set model to eval mode
ResNet50_model.eval()


C:\Users\A\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\A\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to C:\Users\A/.cache\torch\hub\checkpoints\resnet50-0676ba61.pth
100%|█████████████████████████████████████████████████████████████████████████████| 97.8M/97.8M [00:10<00:00, 9.48MB/s]


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

### Pre-processing Data

In [25]:
from PIL import Image
from torchvision import transforms
from tqdm import tqdm

def path_to_tensor(img_path):
    img = Image.open(img_path).convert('RGB') 
    img = img.resize((224, 224))
    
    transform = transforms.ToTensor()
    img_tensor = transform(img)
    
    return img_tensor.unsqueeze(0)  


def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    
    return torch.cat(list_of_tensors) 
    

In [26]:
# Preprocessing input for ResNet50 model
preprocess = transforms.Compose([
    transforms.Resize(256),  # Resize to at least 256
    transforms.CenterCrop(224),  # Center crop to 224x224
    transforms.ToTensor(),  # Convert to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize based on ImageNet
])

def ResNet50_predict_labels(img_path):
    # Load image and apply the preprocessing steps
    img = Image.open(img_path).convert('RGB')
    img = preprocess(img)
    img = img.unsqueeze(0)  # Add batch dimension

    # Move image tensor to the same device as the model (GPU or CPU)
    img = img.to(device)

    # Pass the preprocessed image to the model and get the prediction
    with torch.no_grad():
        output = ResNet50_model(img)

    # Get the predicted label (class with the highest score)
    _, predicted_label = torch.max(output, 1)
    return predicted_label.item()

In [27]:
def predict_breed(img_path, model):
    # Load and preprocess the image
    img_tensor = path_to_tensor(img_path).to(device)
    
    # Disable gradient calculation for inference
    with torch.no_grad():
        output = model(img_tensor)
    
    # Get the predicted class
    _, predicted_label = torch.max(output, 1)
    
    # Check if the predicted class is within dog breed indices (151-268)
    is_dog = 151 <= predicted_label.item() <= 268
    return is_dog, predicted_label.item()


In [28]:
# Load the pre-trained ResNet50 model
ResNet50_model = models.resnet50(pretrained=True).to(device)
ResNet50_model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [29]:
from torch.utils.data import DataLoader
import os
# Assuming validation dataset is stored in a folder 'valid_data' with subfolders for each breed
valid_dir = r'C:\Users\A\Downloads\dogImages\valid'
correct = 0
total = 0

# Loop over the validation dataset
for breed_folder in os.listdir(valid_dir):
    breed_folder_path = os.path.join(valid_dir, breed_folder)
    if not os.path.isdir(breed_folder_path):
        continue  # Skip non-folder files
    
    for img_file in tqdm(os.listdir(breed_folder_path)):
        img_path = os.path.join(breed_folder_path, img_file)
        is_dog, predicted_label = predict_breed(img_path, ResNet50_model)
        
        # Increment correct count if prediction matches a dog breed
        total += 1
        if is_dog:
            correct += 1

# Calculate accuracy
accuracy = correct / total * 100
print(f'Accuracy of the model in detecting dog breeds: {accuracy:.2f}%')


100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 62.97it/s]

Accuracy of the model in detecting dog breeds: 96.41%
